# sqlalchemy ORM

## Connect to database

In [1]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# Declare a Mapping

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users' 
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)  
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

In [3]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

## Create a Schema

In [4]:
Base.metadata.create_all(engine)

2017-04-15 17:15:19,511 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-15 17:15:19,513 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,516 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-15 17:15:19,517 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,520 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-15 17:15:19,521 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,524 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-04-15 17:15:19,525 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,527 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class¶

In [5]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

## Creating a Session

In [6]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

## Adding and Updating Objects¶

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)
our_user = session.query(User).filter_by(name='ed').first()
ed_user is our_user

2017-04-15 17:15:19,588 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-15 17:15:19,593 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 17:15:19,594 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2017-04-15 17:15:19,597 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-04-15 17:15:19,600 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


True

In [8]:
session.commit()

2017-04-15 17:15:19,609 INFO sqlalchemy.engine.base.Engine COMMIT


## Rolling Back

In [9]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)
fake_user in session

True

In [10]:
session.rollback()
fake_user in session

False

In [11]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2017-04-15 17:15:19,656 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-15 17:15:19,670 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2017-04-15 17:15:19,675 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='edspassword')>]

## Query

In [12]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])
session.commit()

2017-04-15 17:15:19,694 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 17:15:19,697 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2017-04-15 17:15:19,700 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 17:15:19,701 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2017-04-15 17:15:19,703 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 17:15:19,705 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2017-04-15 17:15:19,707 INFO sqlalchemy.engine.base.Engine COMMIT


### Query a table

In [13]:
for i in session.query(User).order_by(User.id):
    print(i.name, i.fullname)

2017-04-15 17:15:19,720 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-15 17:15:19,724 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2017-04-15 17:15:19,725 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Query by sepcific columns

In [14]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2017-04-15 17:15:19,775 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2017-04-15 17:15:19,776 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Query the all rows and a column 

In [15]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2017-04-15 17:15:19,793 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-04-15 17:15:19,797 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', password='edspassword')> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar')> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527')> mary
<User(name='fred', fullname='Fred Flinstone', password='blah')> fred


### Change the label

In [16]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2017-04-15 17:15:19,818 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2017-04-15 17:15:19,821 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


### Make aliases

In [17]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2017-04-15 17:15:19,837 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias
2017-04-15 17:15:19,839 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', password='edspassword')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


### Slicing of the query

In [18]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2017-04-15 17:15:19,859 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2017-04-15 17:15:19,861 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>


### `filter_by()`

In [19]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2017-04-15 17:15:19,874 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-04-15 17:15:19,877 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


### `filter()`

In [20]:
for name in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2017-04-15 17:15:19,890 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-04-15 17:15:19,892 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
('ed',)


### `AND` of filters

In [21]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2017-04-15 17:15:19,906 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2017-04-15 17:15:19,909 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', password='edspassword')>


# Building relationship between tables

In [22]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")    
Base.metadata.create_all(engine)

2017-04-15 17:15:19,940 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-15 17:15:19,942 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,944 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2017-04-15 17:15:19,945 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,949 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2017-04-15 17:15:19,950 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 17:15:19,952 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses
jack.addresses = [
    Address(email_address='jack@google.com'),
    Address(email_address='j25@yahoo.com')]
session.add(jack)
session.commit()

2017-04-15 17:15:19,970 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 17:15:19,973 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2017-04-15 17:15:19,978 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2017-04-15 17:15:19,986 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 5)
2017-04-15 17:15:19,988 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2017-04-15 17:15:19,989 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 5)
2017-04-15 17:15:19,990 INFO sqlalchemy.engine.base.Engine COMMIT


jack = session.query(User).filter_by(name='jack').one()
jack

In [25]:
jack.addresses

2017-04-15 17:16:43,833 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2017-04-15 17:16:43,834 INFO sqlalchemy.engine.base.Engine (5,)


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

## Querying with Joins

### Implicit `join`

In [28]:
for u, a in session.query(User, Address).filter(User.id==Address.user_id).all():
    print(u, a)

2017-04-15 17:19:11,691 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id
2017-04-15 17:19:11,693 INFO sqlalchemy.engine.base.Engine ()
<User(name='jack', fullname='Jack Bean', password='gjffdd')> <Address(email_address='jack@google.com')>
<User(name='jack', fullname='Jack Bean', password='gjffdd')> <Address(email_address='j25@yahoo.com')>


### Automatic `join` by foreign key

In [31]:
session.query(User, Address).join(Address).all()

2017-04-15 17:21:27,803 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users JOIN addresses ON users.id = addresses.user_id
2017-04-15 17:21:27,804 INFO sqlalchemy.engine.base.Engine ()


[(<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='jack@google.com')>),
 (<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='j25@yahoo.com')>)]

### Explict condition

In [32]:
session.query(User, Address).join(Address, User.id==Address.user_id).all()

2017-04-15 17:22:43,287 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users JOIN addresses ON users.id = addresses.user_id
2017-04-15 17:22:43,288 INFO sqlalchemy.engine.base.Engine ()


[(<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='jack@google.com')>),
 (<User(name='jack', fullname='Jack Bean', password='gjffdd')>,
  <Address(email_address='j25@yahoo.com')>)]